In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("../pca_Data/train_inpca.csv")
val_df = pd.read_csv("../pca_Data/val_inpca.csv")
test_df = pd.read_csv("../pca_Data/test_inpca.csv")

# train_df = pd.read_csv("../pca_Data/train_pca.csv")
# val_df = pd.read_csv("../pca_Data/val_pca.csv")
# test_df = pd.read_csv("../pca_Data/test_pca.csv")





In [3]:
#functions
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [4]:
train_df.tail()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
113837,0.738838,0.769896,0.647618,0.141620,0.548626,0.520238,0.486246,0.717890,0.766339,0.618480,...,0.452435,0.513156,0.455174,0.425523,0.663846,0.229628,0.575255,0.437382,0.469530,0.224934
113838,0.990562,0.772441,0.858606,0.321666,0.547715,0.522188,0.485039,0.718091,0.591949,0.613208,...,0.481904,0.516460,0.455580,0.552098,0.665469,0.404771,0.578567,0.280226,0.470638,0.221202
113839,0.954434,0.772015,0.897153,0.232334,0.547603,0.522499,0.485313,0.718049,0.568655,0.614005,...,0.481097,0.516181,0.455424,0.485373,0.665439,0.466767,0.578035,0.395709,0.470879,0.221069
113840,0.960711,0.771828,0.871183,0.219663,0.547710,0.522642,0.485244,0.717897,0.555382,0.614618,...,0.524574,0.516252,0.455293,0.468254,0.665529,0.427514,0.577500,0.397886,0.470519,0.221163
113841,0.750942,0.771306,0.611340,0.161777,0.548321,0.518908,0.485035,0.718897,0.640961,0.613068,...,0.440769,0.513577,0.456727,0.522774,0.664300,0.338091,0.576972,0.423311,0.469380,0.224903


In [5]:
train_x = train_df[:]
val_x = val_df.drop('Class',axis=1)
val_y = val_df['Class']

In [6]:
from sklearn.ensemble import IsolationForest 
from sklearn.metrics import classification_report,f1_score

val_contamination = val_y.mean()
print(val_contamination)


0.0010540369615627855


In [74]:
colnames = ['V12','V17','V18','V19','V9','V14']
# colnames = val_x.columns

In [22]:
all_cols = list(val_x.columns)

selected_cols = []

deselected_cols = all_cols

AttributeError: 'list' object has no attribute 'max'

In [90]:
from select import select


all_cols = list(val_x.columns)

selected_cols = ['V10','V17']
deselected_cols = all_cols

for col in selected_cols:
    deselected_cols.remove(col)
    

count = 1

maxf1s = []
frame = pd.DataFrame({'len':[0],'colnames':['V0'] ,'f1s':[0]})

for j in range(1,17):
    print('count :' ,count,"번")

    
    length = []
    colnames = []
    f1s = []    
    for col in deselected_cols:
        count+=1
        
        temp_cols = selected_cols.copy()
        temp_cols.append(col)
        
        print("selected",selected_cols)
        print("tempcol",temp_cols)
        print(col)
        
        model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
        model.fit(train_x.loc[:,temp_cols])

        pred = model.predict(val_x.loc[:,temp_cols])
        pred = get_pred_label(pred)
        f1score = f1_score(val_y, pred, average='macro')
        print(classification_report(val_y,pred))
        print(f1score)
        
        length.append(len(temp_cols))
        colnames.append(temp_cols)
        f1s.append(f1score)

    temp_df = pd.DataFrame({'len':length,'colnames':colnames ,'f1s':f1s})
    frame = pd.concat([frame,temp_df],axis=0)

    for i in range(len(f1s)):
        if(f1s[i] == max(f1s)):
            print('추가')
            temp_index = i
            selected_cols.append(colnames[i][-1])
            deselected_cols.remove(colnames[i][-1])
            break
    
    
    

count : 1 번
selected ['V10', 'V17']
tempcol ['V10', 'V17', 'V1']
V1


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.72      0.60      0.65        30

    accuracy                           1.00     28462
   macro avg       0.86      0.80      0.83     28462
weighted avg       1.00      1.00      1.00     28462

0.8271056766827749
selected ['V10', 'V17']
tempcol ['V10', 'V17', 'V2']
V2


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.76      0.63      0.69        30

    accuracy                           1.00     28462
   macro avg       0.88      0.82      0.85     28462
weighted avg       1.00      1.00      1.00     28462

0.8453050791372196
selected ['V10', 'V17']
tempcol ['V10', 'V17', 'V3']
V3


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.68      0.57      0.62        30

    accuracy                           1.00     28462
   macro avg       0.84      0.78      0.81     28462
weighted avg       1.00      1.00      1.00     28462

0.8089062742283302
selected ['V10', 'V17', 'V7', 'V18', 'V14', 'V26', 'V8', 'V9', 'V5', 'V24', 'V2', 'V15', 'V12', 'V22', 'V6', 'V19', 'V11']
tempcol ['V10', 'V17', 'V7', 'V18', 'V14', 'V26', 'V8', 'V9', 'V5', 'V24', 'V2', 'V15', 'V12', 'V22', 'V6', 'V19', 'V11', 'V27']
V27


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.63      0.57      0.60        30

    accuracy                           1.00     28462
   macro avg       0.81      0.78      0.80     28462
weighted avg       1.00      1.00      1.00     28462

0.7980433877878794
selected ['V10', 'V17', 'V7', 'V18', 'V14', 'V26', 'V8', 'V9', 'V5', 'V24', 'V2', 'V15', 'V12', 'V22', 'V6', 'V19', 'V11']
tempcol ['V10', 'V17', 'V7', 'V18', 'V14', 'V26', 'V8', 'V9', 'V5', 'V24', 'V2', 'V15', 'V12', 'V22', 'V6', 'V19', 'V11', 'V28']
V28


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.65      0.57      0.61        30

    accuracy                           1.00     28462
   macro avg       0.83      0.78      0.80     28462
weighted avg       1.00      1.00      1.00     28462

0.8033779981712035
추가


In [102]:
frame.sort_values('f1s',ascending=False).head(20)

,len,colnames,f1s
6,7,"[V10, V17, V7, V18, V14, V26, V8]",0.870247
4,9,"[V10, V17, V7, V18, V14, V26, V8, V9, V5]",0.870247
14,4,"[V10, V17, V7, V18]",0.865262
9,13,"[V10, V17, V7, V18, V14, V26, V8, V9, V5, V24,...",0.865262
11,5,"[V10, V17, V7, V18, V14]",0.865262
13,5,"[V10, V17, V7, V18, V16]",0.865262
5,13,"[V10, V17, V7, V18, V14, V26, V8, V9, V5, V24,...",0.865262
20,6,"[V10, V17, V7, V18, V14, V26]",0.863504
5,9,"[V10, V17, V7, V18, V14, V26, V8, V9, V6]",0.863504
1,13,"[V10, V17, V7, V18, V14, V26, V8, V9, V5, V24,...",0.859877


In [86]:
frame=frame.reset_index(drop=True)
frame['colnames'].unique()

TypeError: unhashable type: 'list'

In [89]:
frame3 = frame[:]

In [ ]:
list(frame[frame.f1s == frame.f1s.max()]['colnames'])


In [103]:
frame2.sort_values('f1s', ascending=False)

,len,colnames,f1s
6,2,"[V10, V7]",0.865262
18,7,"[V10, V7, V2, V18, V6, V15, V25]",0.859877
15,2,"[V10, V17]",0.858359
17,7,"[V10, V7, V2, V18, V6, V15, V24]",0.852809
11,5,"[V10, V7, V2, V18, V15]",0.852809
...,...,...,...
22,1,[V23],0.499464
23,1,[V24],0.499437
14,1,[V15],0.499384
12,1,[V13],0.499340


In [111]:
temp_cols = ['V10','V7']
iss = []
f1s = []
for i in range(50,151,5):
    model = IsolationForest(n_estimators=i, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
    model.fit(train_x.loc[:,temp_cols])

    pred = model.predict(val_x.loc[:,temp_cols])
    pred = get_pred_label(pred)
    f1score = f1_score(val_y, pred, average='macro')
    iss.append(i)
    f1s.append(f1score)


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/bas

In [112]:
i_df = pd.DataFrame({'i':iss , 'f1s' : f1s})
i_df

,i,f1s
0,50,0.852809
1,55,0.865262
2,60,0.865262
3,65,0.865262
4,70,0.865262
5,75,0.865262
6,80,0.865262
7,85,0.865262
8,90,0.865262
9,95,0.865262


In [113]:
temp_cols = ['V10','V7']
iss2 = []
f1s2 = []
for i in range(70,130,1):
    model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=0.00001 * i, random_state=42, verbose=0)
    model.fit(train_x.loc[:,temp_cols])

    pred = model.predict(val_x.loc[:,temp_cols])
    pred = get_pred_label(pred)
    f1score = f1_score(val_y, pred, average='macro')
    iss2.append(i)
    f1s2.append(f1score)


/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"
/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/bas

In [114]:
i2_df = pd.DataFrame({'i':iss2 , 'f1s' : f1s2})
i2_df

,i,f1s
0,70,0.826381
1,71,0.839859
2,72,0.839859
3,73,0.839859
4,74,0.839859
5,75,0.839859
6,76,0.839859
7,77,0.839859
8,78,0.839859
9,79,0.839859


In [119]:
model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=0.0012, random_state=42, verbose=0)
model.fit(train_x.loc[:,temp_cols])

pred = model.predict(val_x.loc[:,temp_cols])
pred = get_pred_label(pred)
f1score = f1_score(val_y, pred, average='macro')
print(f1score)
print(classification_report(val_y,pred))

/Users/kte/opt/anaconda3/envs/tensor/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


0.870247282626393
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.83      0.67      0.74        30

    accuracy                           1.00     28462
   macro avg       0.92      0.83      0.87     28462
weighted avg       1.00      1.00      1.00     28462



In [100]:
testcols = ['V10', 'V17', 'V7', 'V18', 'V14', 'V26', 'V8']
model = IsolationForest(n_estimators=100, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
model.fit(train_x.loc[:,testcols])

test_pred = model.predict(test_df.loc[:,testcols]) # model prediction
test_pred = get_pred_label(test_pred)
## Submission
submit = pd.read_csv('../Data/sample_submission.csv')
submit.head()
submit['Class'] = test_pred
submit.to_csv('./submit.csv', index=False)